## We are going to build an object detection model for balloons.

In [2]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Rename the drive path
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [4]:
# Clone darknet
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15399, done.
remote: Total 15399 (delta 0), reused 0 (delta 0), pack-reused 15399
Receiving objects: 100% (15399/15399), 14.01 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (10349/10349), done.


In [5]:
# Set the makefile file
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [6]:
# Compile 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [7]:
# Unzip balloon_dataset.zip
!unzip /mydrive/ObjectDetection/balloon_dataset.zip -d data/

Archive:  /mydrive/ObjectDetection/balloon_dataset.zip
   creating: data/balloon_dataset/
  inflating: data/balloon_dataset/006ff1da108ef307.jpg  
  inflating: data/balloon_dataset/006ff1da108ef307.txt  
  inflating: data/balloon_dataset/00d26f0c2371ccd4.jpg  
  inflating: data/balloon_dataset/00d26f0c2371ccd4.txt  
  inflating: data/balloon_dataset/0322a05cdc4f85be.jpg  
  inflating: data/balloon_dataset/0322a05cdc4f85be.txt  
  inflating: data/balloon_dataset/03998b938998f4a3.jpg  
  inflating: data/balloon_dataset/03998b938998f4a3.txt  
  inflating: data/balloon_dataset/05accc3fd24b21d9.jpg  
  inflating: data/balloon_dataset/05accc3fd24b21d9.txt  
  inflating: data/balloon_dataset/08964a9785b85bbe.jpg  
  inflating: data/balloon_dataset/08964a9785b85bbe.txt  
  inflating: data/balloon_dataset/137524b351cc26f2.jpg  
  inflating: data/balloon_dataset/137524b351cc26f2.txt  
  inflating: data/balloon_dataset/1471991.jpg  
  inflating: data/balloon_dataset/1471991.txt  
  inflating: dat

In [8]:
# Rename yolov3.cfg as yolov3_copy.cfg and copy it to the drive.
!cp cfg/yolov3.cfg /mydrive/ObjectDetection/yolov3_copy.cfg

Set the yolov3_copy file using texteditor as described here,

https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects.


In [8]:
# Copy the file back to the same location
!cp /mydrive/ObjectDetection/yolov3_copy.cfg ./cfg

In [9]:
# Create obj.names and obj.data files
!echo -e 'Balloon' > ./data/obj.names
!echo -e 'classes=1\ntrain=./data/train.txt\nvalid=./data/test.txt\nnames=./data/obj.names\nbackup=/mydrive/ObjectDetection/backup' > ./data/obj.data

In [10]:
# Create train.txt under the data folder
import glob
images_list = glob.glob("data/balloon_dataset/*.jpg")
with open("data/train.txt", "w") as f:
  f.write("\n".join(images_list))

In [11]:
# Download pre-trained weights for the convolutional layers file
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2022-03-06 17:01:08--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2022-03-06 17:01:09--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.3MB/s    in 3.8s    

2022-03-06 17:01:13 (41.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# Start training
!./darknet detector train data/obj.data cfg/yolov3_copy.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.538963), count: 3, class_loss = 0.706992, iou_loss = 0.661635, total_loss = 1.368627 
 total_bbox = 393034, rewritten_bbox = 0.172250 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.751720), count: 6, class_loss = 0.864057, iou_loss = 0.381731, total_loss = 1.245787 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.702383), count: 5, class_loss = 1.077470, iou_loss = 0.597603, total_loss = 1.675072 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.361158), count: 1, class_loss = 0.233355, iou_loss = 0.403312, total_loss = 0.636667 
 total_bbox = 393046, rewritten_bbox = 0.172244 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.638948), count: 9, class_loss = 1.833452, iou_loss = 1.287643, total_loss = 3.121095 
v3 (m

In [13]:
!./darknet detector train data/obj.data cfg/yolov3_copy.cfg /mydrive/ObjectDetection/backup/yolov3_copy_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 449572, rewritten_bbox = 0.207976 % 
]2;1926/2000: loss=0.9 hours left=0.3
 1926: 0.884287, 0.796524 avg loss, 0.000010 rate, 10.394642 seconds, 123264 images, 0.283839 hours left
Loaded: 0.000087 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.787347), count: 3, class_loss = 0.028739, iou_loss = 0.106563, total_loss = 0.135302 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.842339), count: 36, class_loss = 0.608383, iou_loss = 0.659830, total_loss = 1.268213 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.755306), count: 3, class_loss = 0.056183, iou_loss = 0.109217, total_loss = 0.165400 
 total_bbox = 449614, rewritten_bbox = 0.207956 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.827475), count: 7, class_loss = 0.155031, iou_loss = 0.147213, total_loss = 0.302244 
v3 